In [12]:
from pymatgen.core import Structure
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import CrystalNN
import networkx as nx

# 1. Carregar estrutura do arquivo CIF
structure = Structure.from_file("C:/Users/Gabri/Desktop/CIF-TESTE/DB0-m1_o2_bsn_repeat.cif")

# 2. Criar grafo estrutural usando algoritmo de vizinhança
strategy = CrystalNN()  # Algoritmo para encontrar vizinhos
structure_graph = StructureGraph.with_local_env_strategy(structure, strategy)

# 3. Converter para grafo NetworkX
def structure_graph_to_networkx(structure_graph):
    nx_graph = nx.Graph()
    
    # Adicionar nós com atributos atômicos
    for idx, site in enumerate(structure_graph.structure):
        nx_graph.add_node(idx,
                          specie=str(site.specie),
                          atomic_number=site.specie.Z,
                          coords=site.coords)
    
    # Adicionar arestas com atributos de ligação
    for (u, v, data) in structure_graph.graph.edges(data=True):
        nx_graph.add_edge(u, v,
                          bond_length=data["to_jimage"])
    
    return nx_graph

nx_graph = structure_graph_to_networkx(structure_graph)

# 4. Exemplo de uso para ML
print("Número de nós:", nx_graph.number_of_nodes())
print("Número de arestas:", nx_graph.number_of_edges())
print("Atributos do primeiro nó:", nx_graph.nodes[0])
print("Atributos da primeira aresta:", list(nx_graph.edges(data=True))[0])

C:\Users\Gabri\AppData\Local\Temp\ipykernel_7908\1279896024.py:11: FutureWarning: with_local_env_strategy is deprecated, and will be removed on 2025-03-20
Use from_local_env_strategy in pymatgen.analysis.graphs instead.
Deprecated on 2024-03-29.
  structure_graph = StructureGraph.with_local_env_strategy(structure, strategy)


Número de nós: 166
Número de arestas: 192
Atributos do primeiro nó: {'specie': 'Zn', 'atomic_number': 30, 'coords': array([ 5.33391789,  9.81550614, 10.13235953])}
Atributos da primeira aresta: (0, 140, {'bond_length': (0, 0, 0)})


In [13]:
structure_graph

Structure Graph
Structure: 
Structure Summary
Lattice
    abc : 16.805856 24.225209 24.189107
 angles : 84.83803100000002 107.570176 107.696718
 volume : 8944.274796691016
      A : 16.021828035260455 0.0 -5.073245736338625
      B : -7.0341421943509275 23.07880044504785 2.179579019212676
      C : 0.0 0.0 24.189107
    pbc : True True True
PeriodicSite: Zn1 (Zn) (5.334, 9.816, 10.13) [0.5196, 0.4253, 0.4895]
PeriodicSite: Zn2 (Zn) (5.669, 12.86, 10.5) [0.5983, 0.557, 0.5095]
PeriodicSite: Zn3 (Zn) (3.023, 11.72, 9.455) [0.4115, 0.5077, 0.4315]
PeriodicSite: Zn4 (Zn) (3.913, 11.28, 12.38) [0.4589, 0.4889, 0.5641]
PeriodicSite: Zn5 (Zn) (12.56, 0.5754, 18.75) [0.7949, 0.02493, 0.9395]
PeriodicSite: Zn6 (Zn) (6.516, 22.8, -0.4508) [0.8404, 0.9879, 0.06861]
PeriodicSite: Zn7 (Zn) (11.24, 1.688, -2.894) [0.7336, 0.07314, 0.02763]
PeriodicSite: Zn8 (Zn) (3.851, 21.8, 22.68) [0.6552, 0.9448, 0.9898]
PeriodicSite: H1 (H) (12.14, 12.66, 9.729) [0.9986, 0.5487, 0.5622]
PeriodicSite: H2 (H) (9.8

In [27]:

# RETIRAR AS CARGAS PARCIAIS DO ARQUIVO CIF // É NECESSARIO ACESSAR O "DICT" DO ARQUIVO CIF // O VALOR DA CARGA PODE SER VISTO AO ABRIR O CIF COMO .TXT
# PORÉM : O ARQUIVO DE TESTE NÃO VAI TER CARGA PARCIAL, ENTÃO TALVEZ N FAÇA SENTIDO USAR O ESP COMO VAR.
# A NÃO SER Q DE PRA CALCULAR COM O REPEAT. 

parser = CifParser("C:/Users/Gabri/Desktop/CIF-TESTE/DB0-m1_o2_bsn_repeat.cif")
structures = parser.get_structures()
cif_data = parser.as_dict()

cargas_parciais = cif_data['DB0-m1_o2_bsn']['_atom_type_partial_charge']
tipos_atomicos = cif_data['DB0-m1_o2_bsn']['_atom_site_type_symbol']

dataframe={'tipos_atomicos':tipos_atomicos,'cargas_parciais':cargas_parciais}
df=pd.DataFrame(dataframe)
df


C:\Users\Gabri\AppData\Local\Temp\ipykernel_12692\2005047077.py:6: FutureWarning: get_structures is deprecated; use parse_structures in pymatgen.io.cif instead.
The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structures = parser.get_structures()


,tipos_atomicos,cargas_parciais
0,Zn,1.401704
1,Zn,1.389246
2,Zn,1.399771
3,Zn,1.392581
4,O,-1.661816
...,...,...
161,C,-0.170120
162,H,0.146867
163,C,-0.131002
164,H,0.105049


CIF -> GRAPH (PYMATGEN) -> GRAPH(NETWORKX)

Posso salvar os grafos em um arquivo PICKLE e depois usar elas como entrando para os algoritmos.